Para ejercitarte y afianzar lo aprendido sobre **Ajuste de Hiperparametros**, completa los siguientes ejercicios. Recuerda que necesitarás datos que están en el directorio data que acompaña al notebook (búscalo en el repositorio de ejercicios)
  
La solución a los mismos las tienes ya, intenta no mirarlos de primeras pero no dejes de hacerlo cuando hayas terminado por si te pudieran aportar.



### Ejercicio 0

Importa los paquetes y módulos que necesites a lo largo del notebook


In [52]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report, accuracy_score


### Ejercicio 1

Carga el dataset de los diamantes. Este dataset está disponible en seaborn, a través de la función `load_datasets` y el dataset se llama `diamonds`. Visuliza las primeras filas y la información general del dataset.


In [14]:
diamonds = sns.load_dataset("diamonds")

display(diamonds.head())
print()
print(diamonds.info())

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB
None


### Ejercicio 2

En esta ocasión queremos clasificar los diamantes por su tipo de corte (cut) en función del resto de features. Define la variable target y muestra su distribución. ¿Qué tipo de problema queremos resolver?

In [24]:
# Contar cuántos diamantes hay de cada corte
counts = diamonds['cut'].value_counts()
print(counts)

# Problema de clasificación multiclase
# Clases desbalanceadas (Ideal y Premium dominan)

cut
Ideal        21551
Premium      13791
Very Good    12082
Good          4906
Fair          1610
Name: count, dtype: int64


### Ejercicio 3

Deshazte de las filas que tengan un target de menor representación de forma que el resto quede más equilibrado [Esto no es una práctica habitual, se hace para simplificar el ejercicio]

In [26]:
# Filtrar cortes con más de 5000 ejemplos
cortes_permitidos = counts[counts > 5000].index
diamonds_filtrado = diamonds[diamonds['cut'].isin(cortes_permitidos)]

# Verificar que se han eliminado los menos representados
display(diamonds_filtrado['cut'].value_counts())

cut
Ideal        21551
Premium      13791
Very Good    12082
Good             0
Fair             0
Name: count, dtype: int64

### Ejercicio 4

Separa el dataset en train y test. Y haz una lista con las posibles features categóricas y las posibles features numéricas


In [27]:
# Definir X e y
X = diamonds_filtrado.drop('cut', axis=1)  # Features
y = diamonds_filtrado['cut']               # Target

# Separar en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [29]:
# Features categóricas
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Features numéricas
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categóricas:", categorical_features)
print("Numéricas:", numerical_features)


Categóricas: ['color', 'clarity']
Numéricas: ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']


### Ejercicio 5

Haz un one-hot encoding (como prefieras, con sklearn o con pandas) a las variables categóricas. Aplícalo ya a train y test

In [31]:
# Aplicar one-hot encoding solo a las variables categóricas
X_train_encoded = pd.get_dummies(X_train, columns=categorical_features, drop_first=False)
X_test_encoded = pd.get_dummies(X_test, columns=categorical_features, drop_first=False)

# Asegurarse de que test y train tengan las mismas columnas
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

display(X_train_encoded.head())


,carat,depth,table,price,x,y,z,color_D,color_E,color_F,...,color_I,color_J,clarity_IF,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
32810,0.34,62.3,55.0,805,4.45,4.48,2.78,True,False,False,...,False,False,False,False,False,False,True,False,False,False
47621,0.55,61.6,55.0,1881,5.32,5.29,3.27,False,False,False,...,False,False,False,False,False,True,False,False,False,False
20657,1.05,60.7,55.0,8946,6.60,6.65,4.02,False,False,False,...,False,False,False,True,False,False,False,False,False,False
23373,0.33,62.1,58.0,631,4.43,4.40,2.74,True,False,False,...,False,False,False,False,False,False,False,False,True,False
25391,1.50,61.7,54.0,14092,7.28,7.31,4.50,False,False,True,...,False,False,False,False,False,False,True,False,False,False


### Ejercicio 6

Crea los juegos X,y de train y test.

### Ejercicio 7

Instancia una regresión logística y un árbol de decisión para clasificación. La primera con el hiperparámetro max_iter a 10000 y el segundo con los hiperparámetros por defecto. ¿Qué profundidad puede alcanzar el árbol de decisión?

In [32]:
# Regresión logística con max_iter=10000
log_reg = LogisticRegression(max_iter=10000, random_state=42)

# Árbol de decisión con hiperparámetros por defecto
tree_clf = DecisionTreeClassifier(random_state=42)


In [36]:
from sklearn.preprocessing import StandardScaler

# Seleccionar solo columnas numéricas
num_features = numerical_features  # ['carat','depth','table','price','x','y','z']

scaler = StandardScaler()
X_train_encoded[num_features] = scaler.fit_transform(X_train_encoded[num_features])
X_test_encoded[num_features] = scaler.transform(X_test_encoded[num_features])


### Ejercicio 8

Entrena los dos modelos y compara sus classification_report respecto a las predicciones de test.


In [39]:
# Regresión logística
log_reg.fit(X_train_encoded, y_train)

# Árbol de decisión
tree_clf.fit(X_train_encoded, y_train)


DecisionTreeClassifier(random_state=42)

In [40]:
y_pred_log = log_reg.predict(X_test_encoded)
y_pred_tree = tree_clf.predict(X_test_encoded)


In [43]:
print("=== Regresión Logística ===")
print(classification_report(y_test, y_pred_log))

print("=== Árbol de Decisión ===")
print(classification_report(y_test, y_pred_tree))


=== Regresión Logística ===
              precision    recall  f1-score   support

       Ideal       0.76      0.88      0.82      4310
     Premium       0.74      0.72      0.73      2758
   Very Good       0.59      0.44      0.50      2417

    accuracy                           0.72      9485
   macro avg       0.70      0.68      0.68      9485
weighted avg       0.71      0.72      0.71      9485

=== Árbol de Decisión ===
              precision    recall  f1-score   support

       Ideal       0.80      0.79      0.79      4310
     Premium       0.74      0.75      0.75      2758
   Very Good       0.62      0.62      0.62      2417

    accuracy                           0.74      9485
   macro avg       0.72      0.72      0.72      9485
weighted avg       0.74      0.74      0.74      9485



### Ejercicio 9

Define un grid de hiperparámetros y rangos para cada modelo. En el caso de la regresión incluye como mínimo: C, solver y max_iter. En el caso del árbol, incluye como mínimo: max_depth, min_samples_split, min_samples_leaf.

In [55]:
pipe_lr = Pipeline(
    steps=[
        ("model", LogisticRegression(random_state=42))
    ]
)

pipe_tree = Pipeline(
    steps=[
        ("model", DecisionTreeClassifier(random_state=42))
    ]
)


In [48]:
param_grid_lr = {
    "model__C": np.logspace(-3, 3, 20),
    "model__solver": ["lbfgs", "saga"],
    "model__max_iter": [2000, 5000, 10000]
}

param_grid_tree = {
    "model__max_depth": [None, 5, 10, 20, 30],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 5, 10]
}


### Ejercicio 10

Ejecuta una búsqueda aleatoria para cada juego de parámetros y modelo. Muestra los valores escogidos y el score en cada modelo, utiliza un k-fold o cv de 5. Usa como scoring `accuracy` (¿por qué crees que asignamos este scoring?). Emplea un número de iteraciones que consideres conveniente (la tercera parte del total de combinaciones, por ejemplo)

In [49]:
from sklearn.model_selection import RandomizedSearchCV

random_lr = RandomizedSearchCV(
    estimator=pipe_lr,
    param_distributions=param_grid_lr,
    n_iter=30,
    cv=5,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1
)

random_lr.fit(X_train_encoded, y_train)

h:\RepoBootcamp\RepoEjercicios\venv\Lib\site-packages\numpy\ma\core.py:2896: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('model',
                                              LogisticRegression(random_state=42))]),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'model__C': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03]),
                                        'model__max_iter': [2000, 5000, 10000],
                                        'model__solver': ['lbfgs', 'saga']},
                   random_state=42, scoring='accuracy')

In [50]:
print("Mejores hiperparámetros (Logistic Regression):")
print(random_lr.best_params_)

print("Mejor accuracy CV:")
print(random_lr.best_score_)


Mejores hiperparámetros (Logistic Regression):
{'model__solver': 'lbfgs', 'model__max_iter': 10000, 'model__C': np.float64(1.438449888287663)}
Mejor accuracy CV:
0.7226338764828082


In [56]:
random_tree = RandomizedSearchCV(
    estimator=pipe_tree,
    param_distributions=param_grid_tree,
    n_iter=30,
    cv=5,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1
)

random_tree.fit(X_train_encoded, y_train)


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('model',
                                              DecisionTreeClassifier(random_state=42))]),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'model__max_depth': [None, 5, 10, 20,
                                                             30],
                                        'model__min_samples_leaf': [1, 2, 5,
                                                                    10],
                                        'model__min_samples_split': [2, 5, 10,
                                                                     20]},
                   random_state=42, scoring='accuracy')

In [57]:
print("Mejores hiperparámetros (Decision Tree):")
print(random_tree.best_params_)

print("Mejor accuracy CV:")
print(random_tree.best_score_)


Mejores hiperparámetros (Decision Tree):
{'model__min_samples_split': 2, 'model__min_samples_leaf': 10, 'model__max_depth': None}
Mejor accuracy CV:
0.7864465783278405


### Ejercicio 11

Realiza ahora una búsqueda completa (puede que tengas que leer un rato, o ver una pelí o dar un paseo) para modelo. Muestra el classification_report del mejor modelo encontrado por el grid search

### Ejercicio 12

Parece que a la regresión logística no le está sentando bien el dataset, ¿se te ocurre algo que no hayamos hecho que puede que mejore sus resultados?